# Compression – Load Data and compress vectors

## Connect to Weaviate

In [ ]:
import weaviate

client = weaviate.connect_to_local()

client.is_ready()

## Create Collection with PQ configuration

[Docs: Product Quantization (PQ)](https://weaviate.io/developers/weaviate/configuration/compression/pq-compression)

> Note: Product Quantization includes a training phase, which is required to create codebooks (codebooks are used to generate centroids for compressed vectors).<br/>
> In other words, based on your data, it figures out how to best compress your vectors.
>
> The compression training starts when the collection reaches `training_limit` number of objects.<br/>
> Before that, the vectors remain uncompressed, and search happens on uncompressed vectors.

In [ ]:
from weaviate.classes.config import Configure

client.collections.delete("WikiPQ")

# Create a collection here - with Cohere as a vectorizer
client.collections.create(
    name="WikiPQ",

    vectorizer_config=[
        Configure.NamedVectors.text2vec_openai(
            name="main_vector",

            model="text-embedding-3-small",
            source_properties=['title', 'text'],

            # Configure PQ
            vector_index_config=Configure.VectorIndex.hnsw(
                quantizer=Configure.VectorIndex.Quantizer.pq(
                    segments=256, # 1536/6 # new number of dimension segments
                    training_limit=10_000  # (default 100k) number of objects needed to train the codebook
                )
            ),
        )
    ],
)

## The rest is the same

In [ ]:
# from data_loader import import_wiki_data
# import_wiki_data(client, "WikiPQ", 25000)

In [ ]:
# wikiPQ = client.collections.get("WikiPQ")
# wikiPQ.aggregate.over_all()

## Clean up

In [ ]:
# client.collections.delete("WikiPQ")

## Close the client

In [ ]:
client.close()